In [15]:
from numpy import sqrt
from numpy import loadtxt
import numpy as np
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt


dataset = loadtxt('pv_large_4d_sample.csv', delimiter=',')
feature_names = ['d1', 'd2', 'd3', 'd4', 'eta']


#df.columns = feature_names
#print(df.head())

#df = df.rename(columns={'eta': 'efficiency'})
#print(df.describe())


#Split into features and target (Price)
#X = df.drop('efficiency', axis = 1)
#y = df['efficiency']
X = dataset[:,0:4]
y = dataset[:,4]

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
######################################################################

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

############################################################
### Decision tree
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)




DecisionTreeRegressor()

In [16]:
dataset2 = loadtxt('pv_4_layer.csv', delimiter=',')

X_test = dataset2[:,0:4]/200.
y_expected = dataset2[:,4]

predictions = tree.predict(X_test)

print(len(y_expected))
#for i in range(0, len(y_expected)):
#    error += np.abs(predictions[i]-y_expected[i])


40000


In [3]:
import timeit
%timeit tree.predict(X_test)


1.73 ms ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
print(1.74e-3/40000)
print(30.7e-6/40000)

4.35e-08
7.675e-10


In [17]:
error=0.
print(len(y_expected))
idx = 0
for i in range(1, 199):
    for j in range(1, 199):
        idx_i_f = (i+1)*200 + j
        idx_i_b = (i-1)*200 + j
        idx_j_f = i*200 + j + 1
        idx_j_b = i*200 + j - 1
        real_grad_j = (y_expected[idx_j_f] - y_expected[idx_j_b])/2e-9
        real_grad_i = (y_expected[idx_i_f] - y_expected[idx_i_b])/2e-9
        norm = np.sqrt(real_grad_j **2 + real_grad_i ** 2)
        nn_grad_j = (predictions[idx_j_f] - predictions[idx_j_b])/2e-9
        nn_grad_i = (predictions[idx_i_f] - predictions[idx_i_b])/2e-9
        error += (real_grad_j * nn_grad_j + real_grad_i * nn_grad_i) / norm ** 2
        #error +=  (np.abs(real_grad_j - nn_grad_j) + np.abs(real_grad_i - nn_grad_i))  / norm
print(error/len(y_expected))
     

40000
0.8958659623012598


In [11]:
exact_max = [0, 0, 0,0,0]
rf_max = [0, 0, 0, 0, 0]
idx=0
for i in range(0, len(y_expected)):
    if y_expected[idx]>exact_max[4]:
        exact_max[:4] = X_test[idx]*200
        exact_max[4] = y_expected[idx]
    if predictions[idx]>rf_max[4]:
        rf_max[:4] = X_test[idx]*200
        rf_max[4] = predictions[idx]
    idx += 1



In [14]:
print(np.array(exact_max)-np.array(rf_max))
print(rf_max)

[ 0.00000000e+00  0.00000000e+00  1.10000000e+01 -3.00000000e+00
  5.74994505e-05]
[4.0, 49.0, 14.000000000000002, 10.0, 0.08602758389050577]
